# Greenways and Trails

This is an indicator for the 2023 base blueprint.

Created by Amy Keister, last run by Amy Keister on 15 September, 2023. It took 6 hours and 44 minutes to run.

In [1]:
import os
import arcpy
import numpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Greenw\Greenw.gdb"

In [4]:
# define final outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Greenw\GreenwaysAndTrails.tif"

In [5]:
# define sub-indicator outputs to help with user support

In [6]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [7]:
# define additional inputs
# OpenStreetMap highway IN ('footway', 'cycleway', 'bridleway', 'path')
trails= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Greenw\OpenStreetMap20230227\Greenways.gpkg\main.NotPrivateTrails"
sw= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Greenw\OpenStreetMap20230227\Greenways.gpkg\main.Sidewalks"
# impervious surface
imp= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_impervious_2019_release_L48_all_files_20210604\nlcd_2019_impervious_l48_20210604\nlcd_2019_impervious_l48_20210604.img"

### Start Analysis

In [8]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [9]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Greenw\Greenw.gdb


### Identify trails that we believe are connected
There are gaps in the trails that are due to digitizing errors. If a trail is within 1 meter of another trail, we are assuming this is a digitizing error and not a real gap in the trail.

In [20]:
# convert all non-private trails to a geodatabase
with arcpy.EnvManager(extent=Rextent):
    arcpy.management.FeatureToLine(trails, "NotPrivateTrails", None, "ATTRIBUTES")

In [21]:
# reproject trails
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures("NotPrivateTrails", "Trails1", '', None, None, None)

In [22]:
# buffer the trails by 1 meters
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Buffer("Trails1", "TrailsBuff", "1 Meters", "FULL", "ROUND", "NONE", [], "PLANAR")

In [23]:
# dissolve the buffered trails. Should be able to do this as a part of the function above, but it tends to crash so
# I'm doing it seperately
arcpy.management.Dissolve("TrailsBuff", "TrailsBuffDis", [], [], "SINGLE_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Greenw\\Greenw.gdb\\TrailsBuffDis'>

In [24]:
# add and calculate a field to hold unique IDs for the buffered dissolved lines. This will be used to indentify trails
# that are connected
arcpy.management.CalculateField("TrailsBuffDis", "ID", "!OBJECTID!", "PYTHON3", "", "LONG")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Greenw\\Greenw.gdb\\TrailsBuffDis'>

In [25]:
# use a spatial join to attach the unique id from the buffered trails back to the original linework
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.SpatialJoin("Trails1", "TrailsBuffDis", "Trails2", "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "INTERSECT", "", "")

In [26]:
# dissolve trails to find length of connected trails
arcpy.management.Dissolve("Trails2", "Trails2Dis", "ID", "", "MULTI_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Greenw\\Greenw.gdb\\Trails2Dis'>

In [27]:
# add and calculate length field
arcpy.management.CalculateGeometryAttributes("Trails2Dis", "LengthM LENGTH", "METERS", '', sr, "SAME_AS_INPUT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Greenw\\Greenw.gdb\\Trails2Dis'>

In [28]:
# add and calculate length field
#arcpy.management.CalculateField("Trails2Dis", "LengthM", "!shape.length@meters!", "PYTHON3", "", "DOUBLE")

In [29]:
# create a raster based on the trail length value
arcpy.conversion.PolylineToRaster("Trails2Dis", "LengthM", "TrailLength", "MAXIMUM_LENGTH", "NONE", Rextent)

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Greenw\\Greenw.gdb\\TrailLength'>

In [30]:
# make an ISNull version of the trails length raster. This will be used later if we want to assign 0 to areas that are not trails
out_raster = arcpy.sa.IsNull("TrailLength"); out_raster.save("TrailLengthIsNull")

### Identify trails with names
The data source doesn't distinguish between sidewalks and greenways/trails. To try to make that distinction, we found that in the South Atlantic, lines with names tended to be greenways/trails, while nameless lines tended to be sidewalks bordering roads. This doesn't work as well in other parts of the geography (like Missouri). This also may change over time, even in the South Atlantic, as people continue to update the Open street Map. We need to keep an eye on this. It would be better if the data source distinguished between sidewalks and more nature oriented trails.

In [31]:
# pull out trails with names
#arcpy.analysis.Select("Trails2", "Trails2Names", where_clause="name <> ''")

In [32]:
# create a raster based on trails with names
#arcpy.conversion.PolylineToRaster("Trails2Names", "Join_Count", "TrailWNames", "MAXIMUM_LENGTH", "NONE", SEraster)

In [33]:
# make an ISNull version of the trails with names
#out_raster = arcpy.sa.IsNull("TrailWNames"); out_raster.save("TrailWNamesIsNull")

### Perform analysis to calcuate percent impervious near trails

Nearby impervious: is defined as the average impervious surface within a 300 m radius circle surrounding the path. HOWEVER along that 300 m stretch of trail, we only count the impervious surface within a 45 m buffer on either side of the trail, since pixels nearer the trail have a bigger impact on the greenway/trail experience. So really this is looking forward and behind a person 300 m as they walk on the trail.

Local impervious is defined as the percent impervious surface of the 30 m pixel that intersects the trail. 

In [34]:
# buffer all trails
arcpy.analysis.Buffer("Trails2Dis", "Trails2Dis_45mbuff", "45 Meters", "FULL", "ROUND", "NONE", "", "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Greenw\\Greenw.gdb\\Trails2Dis_45mbuff'>

In [35]:
# extract NLCD within 45 meters around trails
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask(imp, "Trails2Dis_45mbuff"); out_raster.save("Imp45")

In [36]:
# use focal statistics to calculate what we are calling "nearby impervious"
out_raster = arcpy.sa.FocalStatistics("Imp45", "Circle 10 CELL", "MEAN", "DATA"); out_raster.save("FocalStat")

In [37]:
# reduce this down again to the pixels that intersect the trails, since those are the only pixels we are scoring
out_raster = arcpy.sa.ExtractByMask("FocalStat", "Trails2Dis"); out_raster.save("ImpNearby")

In [38]:
# extract impervious surfact pixels that intersect trails
out_raster = arcpy.sa.ExtractByMask(imp, "Trails2Dis"); out_raster.save("ImpLocal")

In [39]:
# calculate mean of nearby and local impervous surface
out_raster= arcpy.sa.CellStatistics(["ImpNearby", "ImpLocal"], "MEAN", "DATA", "SINGLE_BAND"); out_raster.save("mean")

In [40]:
# use a nested conditional statement to classify the mean of the local and nearby impervious surface
out_raster= arcpy.sa.Con(arcpy.Raster("mean") >= 10, 1, arcpy.sa.Con(arcpy.Raster("mean") >1, 2, arcpy.sa.Con(arcpy.Raster("mean") <= 1, 3, 0))); out_raster.save("NatDef")

### Give a lower score trails identifed as sidewalks in the openstreetmap data
This uses the sidewalk tag from open street map

In [41]:
# convert all sidewalks to a geodatabase
with arcpy.EnvManager(extent=Rextent):
    arcpy.management.FeatureToLine(sw, "Sidewalks", None, "ATTRIBUTES")

In [42]:
# reproject sidewalks
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures("Sidewalks", "Sidewalks1", '', None, None, None)

In [43]:
# add and calculate a field to use to convert to raster
arcpy.management.CalculateField("Sidewalks1", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Greenw\\Greenw.gdb\\Sidewalks1'>

In [44]:
# create a raster from sidewalks
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolylineToRaster("Sidewalks1", "raster", "SidewalkR", "MAXIMUM_LENGTH", "NONE", Rextent)

In [45]:
# make an ISNull version of the sidewalks
out_raster = arcpy.sa.IsNull("SidewalkR"); out_raster.save("SidewalkRIsNull")

### Make legend values

In [46]:
# use a nested conditional statement to assign legend values based on the mean impervious and trail length
out_raster= arcpy.sa.Con((arcpy.Raster("mean") <= 1) & (arcpy.Raster("TrailLength") >= 40000),7, \
             arcpy.sa.Con(((arcpy.Raster("mean") <= 1) & (arcpy.Raster("TrailLength") >= 5000) | (arcpy.Raster("mean") < 10) & (arcpy.Raster("TrailLength") >= 40000)),6, \
                          arcpy.sa.Con(((arcpy.Raster("mean") <= 1) & (arcpy.Raster("TrailLength") >= 1900) | (arcpy.Raster("mean") < 10) & (arcpy.Raster("TrailLength") >= 5000)|(arcpy.Raster("mean") >= 10) & (arcpy.Raster("TrailLength") >= 40000)),5,\
                                       arcpy.sa.Con(((arcpy.Raster("mean") <= 1) & (arcpy.Raster("TrailLength") < 1900) | (arcpy.Raster("mean") < 10) & (arcpy.Raster("TrailLength") >= 1900)|(arcpy.Raster("mean") >= 10) & (arcpy.Raster("TrailLength") >= 5000)),4,\
                                                    arcpy.sa.Con(((arcpy.Raster("mean") < 10) & (arcpy.Raster("TrailLength") < 1900) | (arcpy.Raster("mean") >= 10) & (arcpy.Raster("TrailLength") >= 1900)),3,\
                                                                 arcpy.sa.Con(((arcpy.Raster("mean") >= 10) & (arcpy.Raster("TrailLength") < 1900)),2,1)))))) \
; out_raster.save("Part1")

In [47]:
# Step one of doing zero/NoData Reclassify to make NoData 1, this will let me to a time next
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Reclassify("Part1", "Value", "NODATA 1;2 2;3 3;4 4;5 5;6 6", "DATA"); out_raster.save("Part2")

In [48]:
# give all areas covered by impervious surface raster a 1 and everything else nodata
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    #out_raster = arcpy.sa.Con(imp, 1, None, "OID <> 127"); out_raster.save("map")
    out_raster = arcpy.sa.Reclassify(imp, "Value", "0 100 1;101 127 NODATA", "DATA"); out_raster.save("map")

In [49]:
# Step two of doing zero/NoData
out_raster= arcpy.sa.Times("Part2", "map"); out_raster.save("Part3")

In [50]:
# Set three of doing zero/NoData. Reclassify to make zeros
out_raster = arcpy.sa.Reclassify("Part3", "Value", "1 0;2 2;3 3;4 4;5 5;6 6", "DATA"); out_raster.save("Part4")

In [51]:
# make trails that are tagged as sidewalks a value of 1, regardless of impervious
out_raster= arcpy.sa.Con("SidewalkRIsNull", "Part4", "1", "Value = 1"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent

In [14]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [15]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", Out , '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [1]:
Out= r"D:\SE_Blueprint_2023\5_IndicatorLegendColorFix\GreenwaysAndTrails.tif"

In [2]:
# set code block for next step

codeblock = """
def Reclass(v):
	if v == 7:
		return '7 = Mostly natural and connected for ≥40 km'
	elif v == 6:
		return '6 = Mostly natural and connected for 5 to <40 km or partly natural and connected for ≥40 km'
	elif v == 5:
		return '5 = Mostly natural and connected for 1.9 to <5 km, partly natural and connected for 5 to <40 km, or developed and connected for ≥40 km'
	elif v == 4:
		return '4 = Mostly natural and connected for <1.9 km, partly natural and connected for 1.9 to <5 km, or developed and connected for 5 to <40 km'
	elif v == 3:
		return '3 = Partly natural and connected for <1.9 km or developed and connected for 1.9 to <5 km'
	elif v == 2:
		return '2 = Developed and connected for <1.9 km'
	elif v == 1:
		return '1 = Sidewalk'
	elif v == 0:
		return '0 = Not identified as a trail, sidewalk, or other path'
"""

In [3]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_IndicatorLegendColorFix\\GreenwaysAndTrails.tif'>

In [12]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 7:
		return 0
	if Value == 6:
		return 105
	if Value == 5:
		return 162
	if Value == 4:
		return 201
	if Value == 3:
		return 225
	if Value == 2:
		return 234
	if Value == 1:
		return 226
	else:
		return 255
		
def Reclass2(Value):
	if Value == 7:
		return 24
	if Value == 6:
		return 2
	if Value == 5:
		return 17
	if Value == 4:
		return 65
	if Value == 3:
		return 116
	if Value == 2:
		return 169
	if Value == 1:
		return 226
	else:
		return 255
		
def Reclass3(Value):
	if Value == 7:
		return 137
	if Value == 6:
		return 141
	if Value == 5:
		return 137
	if Value == 4:
		return 118
	if Value == 3:
		return 75
	if Value == 2:
		return 22
	if Value == 1:
		return 38
	else:
		return 255
"""

In [13]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'indicator'>

In [ ]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)